In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

from torch.utils.data import TensorDataset, DataLoader, random_split

C:\Users\yuche\anaconda3\envs\waterforecast\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("CUDA is available:", torch.cuda.is_available())
print("Number of GPUs available:", torch.cuda.device_count())
if torch.cuda.device_count() > 0:
    print("GPU Properties:", torch.cuda.get_device_properties(0))

PyTorch version: 1.12.1
CUDA version: 11.3
CUDA is available: True
Number of GPUs available: 1
GPU Properties: _CudaDeviceProperties(name='NVIDIA GeForce RTX 2060', major=7, minor=5, total_memory=6143MB, multi_processor_count=30)


In [3]:
data = pd.read_csv('train_processed.csv')
data['issue_date'] = pd.to_datetime(data['issue_date'])
data['month'] = data['issue_date'].dt.month
data['year'] = data['issue_date'].dt.year
cols = ['usgs_rw','mnf_rw','mjo_rw','nino_rw','oni_rw','pdo_rw','pna_rw','soi_rw']
def string_to_float_list(s):
    items = s.strip('[]').split(',')
    return [float(item.strip()) for item in items if item]
for col in cols:
    data[col] = data[col].apply(string_to_float_list)
data

,site_id,year,volume,issue_date,volume_scaled,usgs_rw,mnf_scaled,mnf_rw,pred_mar,pred_apr,pred_may,pred_jun,pred_jul,mjo_rw,nino_rw,oni_rw,pdo_rw,pna_rw,soi_rw,month
0,american_river_folsom_lake,1990,532.100,1990-01-01,0.085628,"[0.0028622252420492066, 0.002893983947030845, ...",0.097819,"[0.03172846549634365, 0.04688166447469881, 0.0...",0.114502,0.125828,0.084614,0.082711,0.031674,"[-0.5, -0.38, 0.47, 0.73, 1.32, -0.03, -0.64, ...","[-0.48, -0.23, 0.0, -0.13, -1.25, -1.15, -0.7,...","[-1.85, -1.69, -1.43, -1.08, -0.83, -0.58, -0....","[-1.24, -1.45, -1.24, -0.32, -0.09, 0.11, 0.56...","[-0.72, -1.06, -1.3, -0.54, -0.14, -0.63, -0.1...","[1.5, 1.2, 1.1, 1.6, 1.2, 0.7, 0.9, -0.3, 0.5,...",1
1,american_river_folsom_lake,1990,532.100,1990-01-08,0.085628,"[0.0027386981453826754, 0.0026116633254561206,...",0.097819,"[0.03172846549634365, 0.04688166447469881, 0.0...",0.114502,0.125828,0.084614,0.082711,0.031674,"[0.47, 0.73, 1.32, -0.03, -0.64, -1.22, -0.63,...","[-0.48, -0.23, 0.0, -0.13, -1.25, -1.15, -0.7,...","[-1.85, -1.69, -1.43, -1.08, -0.83, -0.58, -0....","[-1.24, -1.45, -1.24, -0.32, -0.09, 0.11, 0.56...","[-0.72, -1.06, -1.3, -0.54, -0.14, -0.63, -0.1...","[1.5, 1.2, 1.1, 1.6, 1.2, 0.7, 0.9, -0.3, 0.5,...",1
2,american_river_folsom_lake,1990,532.100,1990-01-15,0.085628,"[0.0022214959691747344, 0.0021420992067206373,...",0.097819,"[0.03172846549634365, 0.04688166447469881, 0.0...",0.114502,0.125828,0.084614,0.082711,0.031674,"[0.73, 1.32, -0.03, -0.64, -1.22, -0.63, 0.46,...","[-0.48, -0.23, 0.0, -0.13, -1.25, -1.15, -0.7,...","[-1.85, -1.69, -1.43, -1.08, -0.83, -0.58, -0....","[-1.24, -1.45, -1.24, -0.32, -0.09, 0.11, 0.56...","[-0.72, -1.06, -1.3, -0.54, -0.14, -0.63, -0.1...","[1.5, 1.2, 1.1, 1.6, 1.2, 0.7, 0.9, -0.3, 0.5,...",1
3,american_river_folsom_lake,1990,532.100,1990-01-22,0.085628,"[0.0018314828817957956, 0.0018156035293049762,...",0.097819,"[0.03172846549634365, 0.04688166447469881, 0.0...",0.114502,0.125828,0.084614,0.082711,0.031674,"[1.32, -0.03, -0.64, -1.22, -0.63, 0.46, 0.56,...","[-0.48, -0.23, 0.0, -0.13, -1.25, -1.15, -0.7,...","[-1.85, -1.69, -1.43, -1.08, -0.83, -0.58, -0....","[-1.24, -1.45, -1.24, -0.32, -0.09, 0.11, 0.56...","[-0.72, -1.06, -1.3, -0.54, -0.14, -0.63, -0.1...","[1.5, 1.2, 1.1, 1.6, 1.2, 0.7, 0.9, -0.3, 0.5,...",1
4,american_river_folsom_lake,1990,532.100,1990-02-01,0.085628,"[0.0015774132419426855, 0.0015003400544524373,...",0.061772,"[0.04688166447469881, 0.057554633545957676, 0....",0.114502,0.125828,0.084614,0.082711,0.031674,"[-0.64, -1.22, -0.63, 0.46, 0.56, -0.31, -0.87...","[-0.23, 0.0, -0.13, -1.25, -1.15, -0.7, -0.51,...","[-1.69, -1.43, -1.08, -0.83, -0.58, -0.4, -0.3...","[-1.45, -1.24, -0.32, -0.09, 0.11, 0.56, -0.44...","[-1.06, -1.3, -0.54, -0.14, -0.63, -0.18, -0.2...","[1.2, 1.1, 1.6, 1.2, 0.7, 0.9, -0.3, 0.5, 0.8,...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17467,yampa_r_nr_maybell,2022,680.923,2022-06-22,0.325856,"[0.2350909063075659, 0.19937133555475045, 0.17...",0.155663,"[0.017243066692590858, 0.05901254930924646, 0....",0.017243,0.059013,0.204278,0.155663,0.025318,"[0.74, 0.26, -0.84, 0.42, 0.34, -0.87, -0.7, -...","[-0.05, 0.25, -0.01, 0.18, -0.29, -0.81, -1.53...","[-0.48, -0.38, -0.4, -0.49, -0.67, -0.81, -0.9...","[-1.81, -1.96, -0.94, -1.96, -3.11, -2.75, -2....","[0.67, 0.56, 0.95, 0.44, 1.13, 0.72, -2.56, 1....","[0.4, 1.4, 0.6, 0.8, 0.7, 1.0, 1.5, 0.5, 1.1, ...",6
17468,yampa_r_nr_maybell,2022,680.923,2022-07-01,0.325856,"[0.22335447591735513, 0.19222742140418736, 0.1...",0.025318,"[0.05901254930924646, 0.20427827444061816, 0.1...",0.017243,0.059013,0.204278,0.155663,0.025318,"[0.26, -0.84, 0.42, 0.34, -0.87, -0.7, -0.61, ...","[0.25, -0.01, 0.18, -0.29, -0.81, -1.53, -0.7,...","[-0.38, -0.4, -0.49, -0.67, -0.81, -0.98, -0.9...","[-1.96, -0.94, -1.96, -3.11, -2.75, -2.71, -2....","[0.56, 0.95, 0.44, 1.13, 0.72, -2.56, 1.01, 0....","[1.4, 0.6, 0.8, 0.7, 1.0, 1.5, 0.5, 1.1, 1.8, ...",7
17469,yampa_r_nr_may

# LSTM Model

In [4]:
class MultiLSTM(nn.Module):
    def __init__(self, months, dropout_rate = 0.2):
        super(MultiLSTM, self).__init__()
        # LSTM layers for each input
        self.lstm_usgs = nn.LSTM(30, 32, 2, batch_first = True)
        self.lstm_mnf = nn.LSTM(3, 32, batch_first = True)
        self.lstm_mjo = nn.LSTM(18, 32, 2, batch_first = True)
        self.lstm_nino = nn.LSTM(12, 32, batch_first = True)
        self.lstm_oni = nn.LSTM(12, 32, batch_first = True)
        self.lstm_pdo = nn.LSTM(12, 32, batch_first = True)
        self.lstm_pna = nn.LSTM(12, 32, batch_first = True)
        self.lstm_soi = nn.LSTM(12, 32, batch_first = True)
        self.month = nn.Embedding(months+1, 32)

        # Total concat size
        concat_size = 32*9

        self.dropout = nn.Dropout(dropout_rate)
        
        # Final dense layers after concatenation
        self.fcc_1 = nn.Linear(concat_size, 128)
        self.fcc_2 = nn.Linear(128, 128)
        self.fcc_3 = nn.Linear(128, 64)
        self.fcc_4 = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 3)

    def forward(self, usgs, mnf, mjo, nino, oni, pdo, pna, soi, month):
        # Process each LSTM Input
        usgs_out, _ = self.lstm_usgs(usgs)
        mnf_out, _ = self.lstm_mnf(mnf)
        mjo_out, _ = self.lstm_mjo(mjo)
        nino_out, _ = self.lstm_nino(nino)
        oni_out, _ = self.lstm_oni(oni)
        pdo_out, _ = self.lstm_pdo(pdo)
        pna_out, _ = self.lstm_pna(pna)
        soi_out, _ = self.lstm_soi(soi)
        month_out = self.month(month)
        # Combine all outputs
        combined = torch.cat([usgs_out, mnf_out, mjo_out, nino_out, oni_out, pdo_out, pna_out, soi_out, month_out], dim=1)
        combined = self.dropout(combined)
        # Final dense layers and output
        # Pass through fully connected layers
        # x = torch.relu(self.fcc_1(combined))
        x = self.fcc_1(combined)
        x = self.dropout(x)
        x = torch.relu(self.fcc_2(x))
        x = self.dropout(x)
        x = torch.relu(self.fcc_3(x))
        x = self.dropout(x)
        x = torch.relu(self.fcc_4(x))

        output = self.output_layer(x)
        return output
    def enable_dropout(self):
        """enable montecarlo dropout"""
        for module in self.modules():
            if isinstance(module, nn.Dropout):
                module.train()

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        """
        Args:
            patience (int): How many epochs to wait after last time validation loss improved.
                            Default: 5
            min_delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                                Default: 0
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0


In [10]:
def quantile_loss(preds, target, quantiles = [0.10,0.50,0.90]):
    assert len(quantiles) == preds.shape[1]
    loss = 0
    for i, q in enumerate(quantiles):
        errors = target - preds[:, i]
        loss += torch.max((q-1) * errors, q * errors).mean()
    return loss / len(quantiles)
def train_LSTM(df, pred_month, output_col, num_epochs = 200, model_name = 'model.pth'):
    filtered_df = df[df['month']<=pred_month]
    usgs_tensor = torch.tensor(filtered_df['usgs_rw'].tolist()).float()
    mnf_tensor = torch.tensor(filtered_df['mnf_rw'].tolist()).float()
    mjo_tensor = torch.tensor(filtered_df['mjo_rw'].tolist()).float()
    nino_tensor = torch.tensor(filtered_df['nino_rw'].tolist()).float()
    oni_tensor = torch.tensor(filtered_df['oni_rw'].tolist()).float()
    pdo_tensor = torch.tensor(filtered_df['pdo_rw'].tolist()).float()
    pna_tensor = torch.tensor(filtered_df['pna_rw'].tolist()).float()
    soi_tensor = torch.tensor(filtered_df['soi_rw'].tolist()).float()
    month_tensor = torch.tensor(filtered_df['month'].tolist(), dtype = torch.float32).long()
    target_tensor = torch.tensor(filtered_df[output_col].tolist()).float()
    dataset = TensorDataset(usgs_tensor, mnf_tensor, mjo_tensor, nino_tensor, oni_tensor, pdo_tensor, pna_tensor, soi_tensor, month_tensor, target_tensor)
    
    val_size = int(len(dataset)*0.1)
    train_size = len(dataset) - val_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
    
    model = MultiLSTM(pred_month)
    if torch.cuda.is_available():
        model = model.cuda()
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
    early_stopping = EarlyStopping(patience=5, min_delta=0.0001)
    for epoch in range(num_epochs):
        model.train() # Set model to training mode
        total_loss = 0
        for batch in train_loader:
            usgs, mnf, mjo, nino, oni, pdo, pna, soi, month, target = batch
    
            # Move data to GPU
            if torch.cuda.is_available():
                usgs, mnf, mjo, nino, oni, pdo, pna, soi, month, target = [x.cuda() for x in [usgs, mnf, mjo, nino, oni, pdo, pna, soi, month, target]]
    
            # Zero the gradients
            optimizer.zero_grad()
    
            # Forward pass
            outputs = model(usgs, mnf, mjo, nino, oni, pdo, pna, soi, month)
    
            # Compute loss
            loss = quantile_loss(outputs, target)
    
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
    
            total_loss += loss.item()
        avg_train_loss = total_loss / len(train_loader)
        
        model.eval() # Set model to evaluation mode
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                usgs, mnf, mjo, nino, oni, pdo, pna, soi, month, target = batch
                if torch.cuda.is_available():
                    usgs, mnf, mjo, nino, oni, pdo, pna, soi, month, target = [x.cuda() for x in [usgs, mnf, mjo, nino, oni, pdo, pna, soi, month, target]]
    
                # Forward pass
                outputs = model(usgs, mnf, mjo, nino, oni, pdo, pna, soi, month)
    
                # Calculate loss
                val_loss = quantile_loss(outputs, target)
                total_val_loss += val_loss.item()
    
        avg_val_loss = total_val_loss / len(val_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}')
        early_stopping(avg_val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
    torch.save(model.state_dict(), model_name)

In [11]:
train_LSTM(data, 3, 'pred_mar', model_name = 'mar_modelth')

Epoch [1/200], Training Loss: 0.0152, Validation Loss: 0.0117
Epoch [2/200], Training Loss: 0.0139, Validation Loss: 0.0102
Epoch [3/200], Training Loss: 0.0115, Validation Loss: 0.0082
Epoch [4/200], Training Loss: 0.0104, Validation Loss: 0.0090
Epoch [5/200], Training Loss: 0.0104, Validation Loss: 0.0076
Epoch [6/200], Training Loss: 0.0100, Validation Loss: 0.0078
Epoch [7/200], Training Loss: 0.0097, Validation Loss: 0.0074
Epoch [8/200], Training Loss: 0.0096, Validation Loss: 0.0074
Epoch [9/200], Training Loss: 0.0092, Validation Loss: 0.0071
Epoch [10/200], Training Loss: 0.0090, Validation Loss: 0.0071
Epoch [11/200], Training Loss: 0.0088, Validation Loss: 0.0063
Epoch [12/200], Training Loss: 0.0088, Validation Loss: 0.0063
Epoch [13/200], Training Loss: 0.0086, Validation Loss: 0.0068
Epoch [14/200], Training Loss: 0.0085, Validation Loss: 0.0062
Epoch [15/200], Training Loss: 0.0084, Validation Loss: 0.0064
Epoch [16/200], Training Loss: 0.0080, Validation Loss: 0.0063
E

In [13]:
train_LSTM(data, 4, 'pred_apr', model_name = 'apr_model_quant.pth')

Epoch [1/200], Training Loss: 0.0182, Validation Loss: 0.0148
Epoch [2/200], Training Loss: 0.0141, Validation Loss: 0.0118
Epoch [3/200], Training Loss: 0.0123, Validation Loss: 0.0117
Epoch [4/200], Training Loss: 0.0118, Validation Loss: 0.0109
Epoch [5/200], Training Loss: 0.0113, Validation Loss: 0.0106
Epoch [6/200], Training Loss: 0.0111, Validation Loss: 0.0103
Epoch [7/200], Training Loss: 0.0107, Validation Loss: 0.0100
Epoch [8/200], Training Loss: 0.0106, Validation Loss: 0.0098
Epoch [9/200], Training Loss: 0.0104, Validation Loss: 0.0102
Epoch [10/200], Training Loss: 0.0103, Validation Loss: 0.0096
Epoch [11/200], Training Loss: 0.0102, Validation Loss: 0.0097
Epoch [12/200], Training Loss: 0.0101, Validation Loss: 0.0097
Epoch [13/200], Training Loss: 0.0102, Validation Loss: 0.0097
Epoch [14/200], Training Loss: 0.0100, Validation Loss: 0.0095
Epoch [15/200], Training Loss: 0.0100, Validation Loss: 0.0094
Epoch [16/200], Training Loss: 0.0100, Validation Loss: 0.0096
E

In [14]:
train_LSTM(data, 5, 'pred_may', model_name = 'may_model_quant.pth')

Epoch [1/200], Training Loss: 0.0256, Validation Loss: 0.0201
Epoch [2/200], Training Loss: 0.0199, Validation Loss: 0.0186
Epoch [3/200], Training Loss: 0.0188, Validation Loss: 0.0183
Epoch [4/200], Training Loss: 0.0183, Validation Loss: 0.0181
Epoch [5/200], Training Loss: 0.0180, Validation Loss: 0.0177
Epoch [6/200], Training Loss: 0.0177, Validation Loss: 0.0174
Epoch [7/200], Training Loss: 0.0175, Validation Loss: 0.0177
Epoch [8/200], Training Loss: 0.0174, Validation Loss: 0.0172
Epoch [9/200], Training Loss: 0.0172, Validation Loss: 0.0173
Epoch [10/200], Training Loss: 0.0172, Validation Loss: 0.0170
Epoch [11/200], Training Loss: 0.0170, Validation Loss: 0.0169
Epoch [12/200], Training Loss: 0.0169, Validation Loss: 0.0168
Epoch [13/200], Training Loss: 0.0168, Validation Loss: 0.0171
Epoch [14/200], Training Loss: 0.0167, Validation Loss: 0.0164
Epoch [15/200], Training Loss: 0.0166, Validation Loss: 0.0163
Epoch [16/200], Training Loss: 0.0164, Validation Loss: 0.0162
E

In [15]:
train_LSTM(data, 6, 'pred_jun', model_name = 'jun_model_quant.pth')

Epoch [1/200], Training Loss: 0.0290, Validation Loss: 0.0259
Epoch [2/200], Training Loss: 0.0244, Validation Loss: 0.0236
Epoch [3/200], Training Loss: 0.0226, Validation Loss: 0.0226
Epoch [4/200], Training Loss: 0.0217, Validation Loss: 0.0209
Epoch [5/200], Training Loss: 0.0210, Validation Loss: 0.0208
Epoch [6/200], Training Loss: 0.0205, Validation Loss: 0.0201
Epoch [7/200], Training Loss: 0.0202, Validation Loss: 0.0201
Epoch [8/200], Training Loss: 0.0199, Validation Loss: 0.0196
Epoch [9/200], Training Loss: 0.0198, Validation Loss: 0.0197
Epoch [10/200], Training Loss: 0.0196, Validation Loss: 0.0196
Epoch [11/200], Training Loss: 0.0193, Validation Loss: 0.0193
Epoch [12/200], Training Loss: 0.0193, Validation Loss: 0.0195
Epoch [13/200], Training Loss: 0.0190, Validation Loss: 0.0194
Epoch [14/200], Training Loss: 0.0189, Validation Loss: 0.0192
Epoch [15/200], Training Loss: 0.0189, Validation Loss: 0.0189
Epoch [16/200], Training Loss: 0.0188, Validation Loss: 0.0187
E

In [16]:
train_LSTM(data, 7, 'pred_jul', model_name = 'jul_model_quant.pth')

Epoch [1/200], Training Loss: 0.0140, Validation Loss: 0.0128
Epoch [2/200], Training Loss: 0.0124, Validation Loss: 0.0118
Epoch [3/200], Training Loss: 0.0119, Validation Loss: 0.0112
Epoch [4/200], Training Loss: 0.0116, Validation Loss: 0.0111
Epoch [5/200], Training Loss: 0.0114, Validation Loss: 0.0107
Epoch [6/200], Training Loss: 0.0112, Validation Loss: 0.0105
Epoch [7/200], Training Loss: 0.0110, Validation Loss: 0.0103
Epoch [8/200], Training Loss: 0.0108, Validation Loss: 0.0104
Epoch [9/200], Training Loss: 0.0108, Validation Loss: 0.0102
Epoch [10/200], Training Loss: 0.0107, Validation Loss: 0.0100
Epoch [11/200], Training Loss: 0.0106, Validation Loss: 0.0100
Epoch [12/200], Training Loss: 0.0105, Validation Loss: 0.0099
Epoch [13/200], Training Loss: 0.0105, Validation Loss: 0.0099
Epoch [14/200], Training Loss: 0.0103, Validation Loss: 0.0097
Epoch [15/200], Training Loss: 0.0102, Validation Loss: 0.0097
Epoch [16/200], Training Loss: 0.0102, Validation Loss: 0.0096
E